# Classifier

1. Logistic regression (https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python)


functions:
https://scikit-learn.org/stable/modules/feature_selection.html

example:
https://scikit-learn.org/stable/auto_examples/feature_selection/plot_feature_selection.html#sphx-glr-auto-examples-feature-selection-plot-feature-selection-py

tut:
https://medium.com/@GouthamPeri/pipeline-with-tuning-scikit-learn-b2789dca9dc2

https://stackoverflow.com/questions/33376078/python-feature-selection-in-pipeline-how-determine-feature-names


https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py

https://scikit-learn.org/stable/auto_examples/svm/plot_svm_anova.html#sphx-glr-auto-examples-svm-plot-svm-anova-py

## Import packages

In [70]:
import pandas as pd
from tqdm.notebook import tqdm
from time import time
import random
random.seed(32)
import sklearn
# from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import f1_score, classification_report, precision_recall_curve, roc_auc_score, plot_roc_curve, plot_confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import data

In [2]:
df = pd.read_pickle("wordlists_lin_feat_df_withoutuserfeat_allcomments.pkl")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1606 entries, -Areopagan- to zyzee
Columns: 103888 entries, body to mrc_pmean
dtypes: float64(87790), int64(16092), object(6)
memory usage: 1.2+ GB


In [3]:
# predictors = df.columns.tolist()
# i = 0
# predictorsfile=open('predictorslist.txt','w')
# # predictorsfile.writelines(predictors)
# for element in predictors:
#     predictorsfile.write(str(i))
#     predictorsfile.write(" ")
#     predictorsfile.write(element)
#     predictorsfile.write('\n')
#     i+=1
# predictorsfile.close()

In [4]:
# multilevel columns

lst1 = (5)*["data"]
lst9 = (15-5)*["traits"]
lst10 = ["data"]
lst2 = (21-17)*["global"]
lst3 = (45-21)*["time"]
lst4 = (16103-45)*["subreddits"]
lst5 = (16116-16103)*["extra_features"]
lst6 = (96308-16116)*["word_ngrams"]
lst7 = (103829-96308)*["char_ngrams"]
lst8 = (103889-103829)*["wordlists"]
headers = lst1 + lst9  + lst10 + lst2 + lst3 + lst4 +lst5 + lst6 + lst7 + lst8 
columns = df.columns.values
print(len(headers))
print(len(columns))
arrays = [headers] + [columns]
df.columns=pd.MultiIndex.from_arrays(arrays)

103888
103888


In [6]:
df['traits', 'agree5'] = df['traits', 'agreeableness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
df['traits', 'openn5'] = df['traits', 'openness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
df['traits', 'consc5'] = df['traits', 'conscientiousness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
df['traits', 'extra5'] = df['traits', 'extraversion'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
df['traits', 'neuro5'] = df['traits', 'neuroticism'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))

In [28]:
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')

In [7]:
smalldf = df[['traits', 'global', 'time', 'extra_features', 'word_ngrams', 'char_ngrams', 'wordlists']]
smalldf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1606 entries, -Areopagan- to zyzee
Columns: 87829 entries, ('traits', 'agreeableness') to ('wordlists', 'mrc_pmean')
dtypes: float64(87788), int64(41)
memory usage: 1.1+ GB


In [8]:
n_bins = 20

def hist_true(df, trait):
    fig, ax = plt.subplots()
    plt.hist(df[trait], bins = 20)
    plt.title(trait, y=1.1)
    plt.xlabel("score")

# hist_true(df, "openn")

def all_hist_true(df):
    plt.figure(figsize = (16, 16))
#     plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.subplot(3, 2, 1)
    plt.hist(df['traits', 'openness'], bins = 20)
    plt.title('Agreeableness')
    
    plt.subplot(3, 2, 2)
    plt.hist(df['traits', 'conscientiousness'], bins = 20)
    plt.title('Openness')
    
    plt.subplot(3, 2, 3)
    plt.hist(df['traits', 'extraversion'], bins = 20)
    plt.title('Conscientiousness')
    
    plt.subplot(3, 2, 4)
    plt.hist(df['traits', 'agreeableness'], bins = 20)
    plt.title('Extraversion')
    
    plt.subplot(3, 2, 5)
    plt.hist(df['traits', 'neuroticism'], bins = 20)
    plt.title('Neuroticism')
    
    plt.suptitle("Histograms of the true trait values")
    plt.subplots_adjust(left=0.1, 
                    bottom=0.1,  
                    right=0.9,  
                    top=0.9,  
                    wspace=0.4,  
                    hspace=0.4) 
    plt.show()

## Trait

In [9]:
#split dataset in features and target variable depending on which trait to focus on
def trait(df, classes, trait_name):
    featuredf = df.drop(['data', 'traits'], axis=1, level=0)
    feature_cols = featuredf.columns.tolist()
    
    x = df[feature_cols] 
    
    if classes=='binary':
    
        if trait_name == 'agreeableness':
            y = df['traits', 'agree']
        elif trait_name == 'openness':
            y = df['traits', 'openn']
        elif trait_name == 'conscientiousness':
            y = df['traits', 'consc']
        elif trait_name == 'extraversion':
            y = df['traits', 'extra']
        elif trait_name == 'neuroticism':
            y = df['traits', 'neuro']   
    elif classes=='multi':
        if trait_name == 'agreeableness':
            y = df['traits', 'agree5']
        elif trait_name == 'openness':
            y = df['traits', 'openn5']
        elif trait_name == 'conscientiousness':
            y = df['traits', 'consc5']
        elif trait_name == 'extraversion':
            y = df['traits', 'extra5']
        elif trait_name == 'neuroticism':
            y = df['traits', 'neuro5'] 
    elif classes=='linear':
        if trait_name == 'agreeableness':
            y = df['traits', 'agreeableness']
        elif trait_name == 'openness':
            y = df['traits', 'openness']
        elif trait_name == 'conscientiousness':
            y = df['traits', 'conscientiousness']
        elif trait_name == 'extraversion':
            y = df['traits', 'extraversion']
        elif trait_name == 'neuroticism':
            y = df['traits', 'neuroticism']  
    return x,y 

## Classifier

In [10]:
# ANOVA feature selection for numeric input and categorical output

def create_pipeline(x_train, y_train, classifier, num_feat, weighted):
    if weighted==True: 
        if classifier == "log":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LogisticRegression(class_weight='balanced', max_iter = 200, n_jobs=-1))
            ])
        elif classifier == "multilog":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              
              ('classification', LogisticRegression(class_weight='balanced', multi_class='multinomial', 
                                                    max_iter = 200, solver='lbfgs', n_jobs=-1))
            ])
        elif classifier == "svm":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', svm.SVC(class_weight='balanced', max_iter = 1000))
            ])

    else:
        if classifier == "log":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LogisticRegression(max_iter = 200, n_jobs=-1))
            ])
        elif classifier == "multilog":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LogisticRegression(multi_class='multinomial', max_iter = 200, solver='lbfgs', 
                                                    n_jobs=-1))
            ])
        elif classifier == "mlp":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', MLPClassifier(hidden_layer_sizes=(3,)))
            ])
        elif classifier == "svm":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', svm.SVC(max_iter = 1000))
            ])
        elif classifier == "svmlinear":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', svm.LinearSVC(max_iter = 1000))
            ])
        elif classifier == "knn":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', KNeighborsClassifier(n_neighbors=1, n_jobs=-1))
            ])
        elif classifier == "linear":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LinearRegression(n_jobs=-1))
            ])
    return pipeline

In [56]:
def create_pipeline_cv(X_train, y_train, classifier, num_feat):
    if classifier == "log":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', LogisticRegression())
        ])
    elif classifier == "multilog":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', LogisticRegression(multi_class='multinomial', n_jobs=-1))
        ])
    elif classifier == "mlp":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', MLPClassifier())
        ])
    elif classifier == "svm":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', svm.SVC())
        ])
    elif classifier == "svmlinear":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', svm.LinearSVC())
        ])
    elif classifier == "knn":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', KNeighborsClassifier(n_neighbors=1, n_jobs=-1))
        ])
    elif classifier == "linear":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', LinearRegression(n_jobs=-1))
        ])
    return pipeline

## Get names of 30 most important features

In [12]:
# get names of the features
def get_names(x, pipeline):
    features = pipeline.named_steps['feature_selection']
    names = x.columns[features.get_support(indices=True)]
    return names
# names = get_names(logpipe)
# print(names)

### Get p-values

In [13]:
def get_pvalues(pipeline, x):
#     x_indices = np.arange(x.shape[-1])
#     selector = SelectKBest(f_classif, k=30)
#     selector.fit(x_train, y_train)
#     scores = -np.log10(selector.pvalues_)
    features = pipeline.named_steps['feature_selection']
    pvalues = features.pvalues_
#     pvalues /= pvalues.max()
    dfpvalues = pd.DataFrame(features.pvalues_)
    dfscores = pd.DataFrame(features.scores_)
    dfcolumns = pd.DataFrame(x.columns)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores, dfpvalues],axis=1)
    featureScores.columns = ['specs','score', 'pvalue']
    featureScores.sort_values(by='pvalue')

    plt.figure(figsize = (12, 4))
    plt.subplot(1, 2, 1)
    plt.hist(pvalues, bins=20)
    plt.title('All p-values')
    plt.subplot(1, 2, 2)
    smallpvalues = pvalues[pvalues<0.1]
    plt.hist(smallpvalues, bins=10)
    plt.title('Small p-values')
    
    plt.suptitle("Histograms of the p-values")
    plt.subplots_adjust(left=0.1, 
                    bottom=0.1,  
                    right=0.9,  
                    top=0.9,  
                    wspace=0.4,  
                    hspace=0.4) 
    plt.show()
    
    return featureScores

## Scores

In [14]:
def scores(y_test, y_pred, presentationtype):
    
    if presentationtype == "scores":
        accuracy=metrics.accuracy_score(y_test, y_pred)
        precision=metrics.precision_score(y_test, y_pred)
        recall=metrics.recall_score(y_test, y_pred)
        f_one=metrics.f1_score(y_test, y_pred)
        return accuracy, precision, recall, f_one
    if presentationtype == "report":
        report = classification_report(y_test, y_pred)
        return report


def score_plot(logreg, y_test, x_test):
    lr_probs = logreg.predict_proba(x_test)
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    # predict class values
#     yhat = logreg.predict(x_test)
    lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr_probs)
#     lr_f1, lr_auc = f1_score(y_test, yhat), auc(lr_recall, lr_precision)

    # plot the precision-recall curves
    no_skill = len(y_test[y_test==1]) / len(y_test)
    fig, ax = plt.subplots()
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
    plt.plot(lr_recall, lr_precision, marker='.', label='Classifier')
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    plt.show()
    
    return lr_precision, lr_recall


### Score plot
Smaller values on the x-axis of the plot indicate lower false positives and higher true negatives.


Larger values on the y-axis of the plot indicate higher true positives and lower false negatives.

## Create and print confusion matrix

In [15]:
def create_cnfmatrix(clf, x_test, y_test, y_pred, plotting, detailed):
    cnfpipe_matrix = metrics.confusion_matrix(y_test, y_pred)
#     print(cnfpipe_matrix)
#     disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cnfpipe_matrixcmap=plt.cm.Blues, normalize=normalize)
#     disp.plot() 
    if detailed:
        tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
        sumpositive = tp + fn
        sumnegative = fp + tn
        sumcorrect = tp + tn
        sumwrong = fp + fn
        sumall = tn+fp+fn+tp
        print("TN, FP, FN, TP: ", tn, fp, fn, tp, "\nSum: ", sumall, "\nSum correct predictions: ", 
              sumcorrect, "Percent: ", sumcorrect/sumall, "\nSum wrong predictions: ", sumwrong, "\tPercent: ",
              sumwrong/sumall, "\nSum actual positives: ", sumpositive, "\tPercent: ", sumpositive/sumall,
              "\nSum actual negatives: ", sumnegative, "\tPercent: ", sumnegative/sumall)

    if plotting:
#         %matplotlib inline
#         class_names=[0,1] # name  of classes
#         fig, ax = plt.subplots()
#         tick_marks = np.arange(len(class_names))
#         plt.xticks(tick_marks, class_names)
#         plt.yticks(tick_marks, class_names)
#         # create heatmap
#         sns.heatmap(pd.DataFrame(cnfpipe_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
#         ax.xaxis.set_label_position("bottom")
#         disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cnfpipe_matrix, cmap=plt.cm.Blues, normalize=normalize)
#         disp.plot() 
        plot_confusion_matrix(clf, x_test, y_test,
                                 cmap=plt.cm.Blues,
                                 normalize='true')
#         disp.ax_.set_title('Confusion matrix')
        plt.tight_layout()
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
#         disp.plot() 
        plt.show()
        
# cnfmatrix = create_cnfmatrix(y_test, y_pred, True)

## ROC Curve

In [16]:
def roc_aucscore(clf, x_test, y_test, classes, plotting, detailed):
    if detailed:
        print(roc_auc_score(y, clf.predict_proba(x), multi_class='ovo'))
        return score
    
    if plotting and classes == 'binary':
        plot_roc_curve(clf, x_test, y_test)
        plt.title('ROC Curve', y=1.1)
        plt.show()

# Wrapper


nested stratified cv:
https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html#sphx-glr-auto-examples-model-selection-plot-nested-cross-validation-iris-py

https://weina.me/nested-cross-validation/


https://towardsdatascience.com/nested-cross-validation-hyperparameter-optimization-and-model-selection-5885d84acda
https://gist.github.com/krsatyam1996/9640ed8baa20d3dc11822564710a8d71#file-nested_cv-py

In [32]:
traits = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']

def switching(trait):
    switcher={
            'openness':'all',
            'conscientiousness':'all',
            'agreeableness':'all',
            'extraversion':'all',
            'neuroticism':'all'
         }
    return switcher.get(trait,"Invalid")

def classify(df, classifier, classes, trainscores=False, plotting = False, weighted = False, detailed=False):
    for trait_name in traits:
        num_feat = switching(trait_name)
        print("Trait to predict: ", trait_name)
        x,y = trait(df, classes,trait_name)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
#         scores = cross_val_scores(pipeline,X_train,y_train,cv=5,scoring='f1_macro')
        if trainscores:
            x_test = x_train
            y_test = y_train
        if detailed: 
            print("Number of authors in y_train: ", len(y_train))
            print("Number of authors in y_test: ", len(y_test))
        clf = create_pipeline(x_train, y_train, classifier, num_feat, weighted)
        clf.fit(x_train, y_train)
        y_pred=clf.predict(x_test)
        if classes=='linear':
            print("Score (Reg: r sqared, SVM: accuracy): ", clf.score(x_test, y_test))
#             print(y_pred)    
        else:
            if detailed:
                print("Number of authors in y_pred: ", len(y_pred))
                names = get_names(x, clf)
                print("Names of the top", len(names), "features: \n", names, "\n")
                pvalues = get_pvalues(clf, x)
            #     print("p-values of", len(pvalues), "features: \n", pvalues, "\n")
                if trait_name=="openness":
                    count = pvalues['pvalue'].le(0.02).sum()
                if trait_name=="conscientiousness":
                    count = pvalues['pvalue'].le(0.07).sum()
                if trait_name=="extraversion":
                    count = pvalues['pvalue'].le(0.05).sum()
                if trait_name=="agreeableness":
                    count = pvalues['pvalue'].le(0.04).sum()
                if trait_name=="neuroticism":
                    count = pvalues['pvalue'].le(0.04).sum()
                print("Number of features with this threshold: ", count)
        #         print("\nP-Values: \nNumber of features: ", 30)
        #         print(pvalues.nsmallest(count,'pvalue'))
                print("\n")
            report = scores(y_test, y_pred, "report")
            print("Classification report: \n", report)
            if plotting: 
                cnfmatrix = create_cnfmatrix(clf, x_test, y_test, y_pred, plotting, detailed) 
                rocplot = roc_aucscore(clf, x_test, y_test, classes, plotting, detailed)
        #     accuracy, precision, recall, f_one = scores(y_test, y_pred, "scores")
        #     print("Scores:\nAccuracy:",accuracy, "\nPrecision:",precision, "\nRecall:",recall, "\nF1 score:",f_one)
                if (classifier == 'log' and classes == 'binary'):
                    lr_precision, lr_recall = score_plot(clf, y_test, x_test)
            plt.show()
    #     print("Scores:\nLR_Precision:",lr_precision, "\nLR_Recall:",lr_recall)


## Wrapper with nested stratified cross validation

In [73]:
def get_params(classifier):
    if classifier == 'log':
        params = {'classification__class_weight': [None, 'balanced'], 
                  'classification__solver': ['lbfgs', 'liblinear', 'saga'], 
                  'classification__max_iter': [100, 200, 500, 1000]}
    if classifier == 'multilog':
        params = {'classification__class_weight': [None, 'balanced'], 
                  'classification__solver': ['lbfgs', 'saga'], 
                  'classification__max_iter': [100, 200, 500, 1000]}
    elif classifier == 'mlp':
        params = {'classification__hidden_layer_sizes': [(3,), (5,)]}
    elif classifier == 'svm':
        params = {'classification__gamma': ['scale', 'auto'], 
                  'classification__class_weight': [None, 'balanced'], 
                  'classification__max_iter': [100, 200, 500, 1000]}
    return params


def classify_cv(df, classifier, classes):   
    for trait_name in tqdm(traits):
        num_feat = switching(trait_name)
        print("\n\nTrait to predict: ", trait_name, "\n\n")
        x,y = trait(df, classes, trait_name)
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    
        cv_outer = StratifiedKFold(n_splits=5)
        cv_outer_lst = cv_outer.split(X_train, y_train)

        for train_idx, val_idx in tqdm(cv_outer_lst):
            train_data, val_data = X_train.iloc[train_idx], X_train.iloc[val_idx]
            train_target, val_target = y_train[train_idx], y_train[val_idx]


            print("\n\tCreate pipeline with", classifier, "...")
            clf = create_pipeline_cv(X_train, y_train, classifier, num_feat)
            cv_inner = StratifiedKFold(n_splits=5)
            params = get_params(classifier)
            print("\tStart grid search...")
            t0 = time()
            gd_search = GridSearchCV(clf, params, scoring='f1_macro', n_jobs=-1, cv=cv_inner).fit(train_data, train_target)
            print("\tGrid search done in %0.3fs" % (time() - t0))
            print("\tGet best model...")
            best_model = gd_search.best_estimator_
            print(best_model)

            print("\tFit best model...")
            clfnew = best_model.fit(train_data, train_target)
            y_pred_prob = clfnew.predict_proba(val_data)[:,1]
            y_pred = clfnew.predict(val_data)
            f1_macro = f1_score(val_target, y_pred, average='macro')

            print("Val Acc:",f1_macro , "Best GS Acc:",gd_search.best_score_, "Best Params:",gd_search.best_params_)


#       # Training final model

#     model = LogisticRegression(random_state=7, C=0.001, class_weight='balanced', penalty='l2').fit(X_train, y_train)
#     y_pred_prob = model.predict_proba(X_test)[:,1]
#     print("AUC", metrics.roc_auc_score(y_test, y_pred_prob))
#     print(metrics.confusion_matrix(y_test, y_pred))

In [74]:
classify_cv(df, 'log', 'binary')

  0%|          | 0/5 [00:00<?, ?it/s]



Trait to predict:  openness 




0it [00:00, ?it/s]


	Create pipeline with log ...
	Start grid search...
	 Grid search done in 310.670s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k='all')),
                ('classification', LogisticRegression(solver='liblinear'))])
	Fit best model...
Val Acc: 0.5514646382909857 Best GS Acc: 0.55036819543346 Best Params: {'classification__class_weight': None, 'classification__max_iter': 100, 'classification__solver': 'liblinear'}

	Create pipeline with log ...
	Start grid search...


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Val Acc: 0.5128789841927961 Best GS Acc: 0.49406043837507063 Best Params: {'classification__class_weight': None, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	 Grid search done in 301.933s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k='all')),
                ('classification', LogisticRegression(solver='liblinear'))])
	Fit best model...
Val Acc: 0.5139872795904649 Best GS Acc: 0.5079274925602124 Best Params: {'classification__class_weight': None, 'classification__max_iter': 100, 'classification__solver': 'liblinear'}

	Create pipeline with log ...
	Start grid search...


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


	 Grid search done in 366.299s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k='all')),
                ('classification',
                 LogisticRegression(max_iter=500, solver='saga'))])
	Fit best model...


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Val Acc: 0.4874199093608376 Best GS Acc: 0.5162688685748177 Best Params: {'classification__class_weight': None, 'classification__max_iter': 500, 'classification__solver': 'saga'}


## Histogram of true traits

In [ ]:
def check_imbalance(df, traits):
    length = len(df)
    o = df['traits', 'openn']
    c = df['traits', 'consc']
    e = df['traits', 'extra']
    a = df['traits', 'agree']
    n = df['traits', 'neuro']
    binarylst = [o, c, e, a, n]
    o5 = df['traits', 'openn5']
    c5 = df['traits', 'consc5']
    e5 = df['traits', 'extra5']
    a5 = df['traits', 'agree5']
    n5 = df['traits', 'neuro5']
    multilst = [o5, c5, e5, a5, n5]
    
    result = []
    for trait in binarylst: 
        result.append(np.bincount(trait) / length)
    result5 = []
    for trait in multilst:
        result5.append(np.bincount(trait) / len(trait))
    
    print("Distribution of the true trait values in the classes (in %):\n")
    for i in range(len(traits)):
        print(traits[i], "\n\tBinary: ", result[i], "\n\t5 classes: ", result5[i], "\n")
    
#     result =np.bincount(o) / len(o)
#     result5 =np.bincount(o5) / len(o)
#     print("Openness\n\tBinary: ", result, "\n\t5 classes: ", result5)

    
check_imbalance(df, traits)

In [ ]:
# true histogram plots
all_hist_true(df)

In [ ]:
# %%javascript
# IPython.OutputArea.auto_scroll_threshold = 1000;

## Replicate G&S2018R with k=30 features

### check results on train set with k nearest neighbor algorithm

In [ ]:
classify(df, 'knn', 'binary', trainscores=True, plotting=False, weighted=False, detailed=False)

In [ ]:
classify(df, 'knn', 'multi', trainscores=True, plotting=False, weighted=False, detailed=False)

### Logistic regression

In [ ]:
classify(df, 'log', 'binary', plotting=True)

In [ ]:
classify(df, 'log', 'binary', plotting=True, weighted=True)

### Logistic Regression: multiclass

In [ ]:
classify(df, 'multilog', 'multi', plotting=True, weighted=True, detailed=False)

### MLP

In [ ]:
classify(df, 'mlp', 'binary', plotting=True)

In [ ]:
classify(df, 'mlp', 'multi', plotting=True)

### SVM

In [ ]:
classify(df, 'svm', 'binary', plotting=True)

In [ ]:
classify(df, 'svm', 'multi', plotting=True)

In [ ]:
classify(df, 'svmlinear', 'linear', plotting=True)

### Linear Regression

In [ ]:
classify(df, 'linear', 'linear', plotting=True)

# Results without predictor subreddit

###  binary log (without subreddit)

In [ ]:
classify(smalldf, 'log', 'binary', plotting=True, weighted=True, detailed=False)

### multi log (without subreddit)

In [ ]:
classify(smalldf, 'multilog', 'multi', plotting=True, weighted=True, detailed=False)

### MLP (without subreddit)

In [ ]:
classify(smalldf, 'mlp', 'binary', plotting=True, weighted=False, detailed=False)

### Linear Regression (without subreddit)

In [ ]:
classify(smalldf, 'lin', 'linear', plotting=True)